In [1]:
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
pd.set_option('display.max_columns', 50)

In [4]:
nppes_cols = ['NPI', 
              'Entity Type Code', 
              'Provider First Name',
              'Provider Middle Name',
              'Provider Last Name (Legal Name)', 
              'Provider Name Suffix Text',
              'Provider Credential Text',
              'Provider Organization Name (Legal Business Name)',
              'Provider Other Organization Name',
              'Provider First Line Business Practice Location Address',
              'Provider Second Line Business Practice Location Address',
              'Provider Business Practice Location Address City Name',
              'Provider Business Practice Location Address State Name',
              'Provider Business Practice Location Address Postal Code',
              'Healthcare Provider Taxonomy Code_1',
              'Healthcare Provider Primary Taxonomy Switch_1',
              'Healthcare Provider Taxonomy Code_2',
              'Healthcare Provider Primary Taxonomy Switch_2',
              'Healthcare Provider Taxonomy Code_3',
              'Healthcare Provider Primary Taxonomy Switch_3',
              'Healthcare Provider Taxonomy Code_4',
              'Healthcare Provider Primary Taxonomy Switch_4',
              'Healthcare Provider Taxonomy Code_5',
              'Healthcare Provider Primary Taxonomy Switch_5',
              'Healthcare Provider Taxonomy Code_6',
              'Healthcare Provider Primary Taxonomy Switch_6',
              'Healthcare Provider Taxonomy Code_7',
              'Healthcare Provider Primary Taxonomy Switch_7',
              'Healthcare Provider Taxonomy Code_8',
              'Healthcare Provider Primary Taxonomy Switch_8',
              'Healthcare Provider Taxonomy Code_9',
              'Healthcare Provider Primary Taxonomy Switch_9',
              'Healthcare Provider Taxonomy Code_10',
              'Healthcare Provider Primary Taxonomy Switch_10',
              'Healthcare Provider Taxonomy Code_11',
              'Healthcare Provider Primary Taxonomy Switch_11',
              'Healthcare Provider Taxonomy Code_12',
              'Healthcare Provider Primary Taxonomy Switch_12',
              'Healthcare Provider Taxonomy Code_13',
              'Healthcare Provider Primary Taxonomy Switch_13',
              'Healthcare Provider Taxonomy Code_14',
              'Healthcare Provider Primary Taxonomy Switch_14',
              'Healthcare Provider Taxonomy Code_15',
              'Healthcare Provider Primary Taxonomy Switch_15']

id_cols = ['npi', 
           'entity_type_code', 
           'first_name', 
           'middle_name',
           'last_name',
           'suffix',
           'credentials',
           'organization_name',
           'organization_name_other',
           'address_first_line', 
           'address_second_line', 
           'city',
           'state', 
           'zipcode',
           'cbsa']

var_cols = ['taxonomy_code', 
            'primary_taxonomy_switch']

nppes = pd.read_csv('../data/NPPES_Data_Dissemination_February_2022/npidata_pfile_20050523-20220213.csv',
                    usecols = nppes_cols, 
                    dtype = str, 
                    chunksize = 100000, 
                    low_memory = False)


cbsa = pd.read_excel('../data/ZIP_CBSA_122021.xlsx', dtype = {'zip': str, 'cbsa': str})

regional_cbsa = cbsa.query('cbsa == "34980"') \
    .append(cbsa.query('usps_zip_pref_state in ["AL", "AR", "GA", "MO", "MS", "KY", "NC", "TN", "VA"]') \
            .sort_values('tot_ratio', ascending = False)) \
    .drop_duplicates(subset = 'zip')[['zip', 'cbsa']]

regional_cbsa.columns = ['zipcode', 'cbsa']


nucc = pd.read_csv('../data/nucc_taxonomy_220.csv').drop(['Definition', 'Notes', 'Display Name', 'Section'], axis = 1)

nucc.columns = ['taxonomy_code', 'grouping', 'classification', 'specialization']


regional_npis = pd.DataFrame()

for chunk in tqdm(nppes):

    states = ['AL', 'AR', 'GA', 'KY', 'MO', 'MS', 'NC', 'TN', 'VA']
    filtered_chunk = chunk[nppes_cols].query('`Provider Business Practice Location Address State Name` == @states')

    filtered_chunk.columns = ['npi', 
                              'entity_type_code', 
                              'first_name', 
                              'middle_name',
                              'last_name',
                              'suffix',
                              'credentials',
                              'organization_name',
                              'organization_name_other',
                              'address_first_line', 
                              'address_second_line', 
                              'city',
                              'state', 
                              'zipcode',
                              'taxonomy_code_1',
                              'primary_taxonomy_switch_1',
                              'taxonomy_code_2',
                              'primary_taxonomy_switch_2',
                              'taxonomy_code_3',
                              'primary_taxonomy_switch_3',
                              'taxonomy_code_4',
                              'primary_taxonomy_switch_4',
                              'taxonomy_code_5',
                              'primary_taxonomy_switch_5',
                              'taxonomy_code_6',
                              'primary_taxonomy_switch_6',
                              'taxonomy_code_7',
                              'primary_taxonomy_switch_7',
                              'taxonomy_code_8',
                              'primary_taxonomy_switch_8',
                              'taxonomy_code_9',
                              'primary_taxonomy_switch_9',
                              'taxonomy_code_10',
                              'primary_taxonomy_switch_10',
                              'taxonomy_code_11',
                              'primary_taxonomy_switch_11',
                              'taxonomy_code_12',
                              'primary_taxonomy_switch_12',
                              'taxonomy_code_13',
                              'primary_taxonomy_switch_13',
                              'taxonomy_code_14',
                              'primary_taxonomy_switch_14',
                              'taxonomy_code_15',
                              'primary_taxonomy_switch_15']
    
    filtered_chunk['zipcode'] = filtered_chunk['zipcode'].astype(str).str[:5]

    filtered_chunk = filtered_chunk.merge(regional_cbsa, on = 'zipcode')

    filtered_chunk = pd.wide_to_long(filtered_chunk, stubnames = var_cols, i = id_cols, j = '', sep = '_') \
        .sort_values('npi') \
        .query('primary_taxonomy_switch == "Y"') \
        .reset_index() \
        .drop(['', 'primary_taxonomy_switch'], axis = 1)

    filtered_chunk = filtered_chunk.merge(nucc, on = 'taxonomy_code').drop(columns = 'taxonomy_code')

    regional_npis = regional_npis.append(filtered_chunk)


regional_npis.to_csv('../data/regional_npis.csv', index = False)

0it [00:00, ?it/s]

In [5]:
nashville_facilities = regional_npis \
    .query('entity_type_code == "2" & cbsa == "34980"') \
    .drop(columns = ['first_name', 'middle_name', 'last_name', 'suffix', 'credentials'])

nashville_facilities.to_csv('../data/nashville_facilities.csv', index = False)

In [6]:
hop_teaming = pd.read_csv('../data/DocGraph_Hop_Teaming_2018_Commercial/DocGraph_Hop_Teaming_2018.csv',
                          dtype = {'from_npi': str, 'to_npi': str},
                          chunksize = 1000000,
                          low_memory = False)


entity_type_dict = dict(regional_npis[['npi', 'entity_type_code']].values)
cbsa_dict = dict(regional_npis[['npi', 'cbsa']].values)


hop_teaming_provider_to_facility = pd.DataFrame()

for chunk in tqdm(hop_teaming):
    filtered_chunk = chunk[(chunk['from_npi'].map(entity_type_dict) == '1') & 
                           (chunk['to_npi'].map(entity_type_dict) == '2') &
                           (chunk['to_npi'].map(cbsa_dict) == '34980') &
                           (chunk['transaction_count'] >= 50) &
                           (chunk['average_day_wait'] < 50)]
    hop_teaming_provider_to_facility = hop_teaming_provider_to_facility.append(filtered_chunk)


hop_teaming_provider_to_facility.to_csv('../data/hop_teaming_provider_to_facility.csv', index = False)

0it [00:00, ?it/s]

In [ ]:
hop_teaming_facilities = nashville_facilities[nashville_facilities['npi'].isin(hop_teaming_provider_to_facility['to_npi'])]

hop_teaming_facilities.to_csv('../data/hop_teaming_facilities.csv', index = False)

In [ ]:
hop_teaming = pd.read_csv('../data/DocGraph_Hop_Teaming_2018_Commercial/DocGraph_Hop_Teaming_2018.csv',
                          dtype = {'from_npi': str, 'to_npi': str},
                          chunksize = 1000000,
                          low_memory = False)


entity_type_dict = dict(regional_npis[['npi', 'entity_type_code']].values)
cbsa_dict = dict(regional_npis[['npi', 'cbsa']].values)


hop_teaming_provider_to_provider = pd.DataFrame()

for chunk in tqdm(hop_teaming):
    filtered_chunk = chunk[(chunk['from_npi'].map(entity_type_dict) == '1') & 
                           (chunk['to_npi'].map(entity_type_dict) == '1') &
                           (chunk['to_npi'].map(cbsa_dict) == '34980') &
                           (chunk['transaction_count'] >= 50) &
                           (chunk['average_day_wait'] < 50)]
    hop_teaming_provider_to_provider = hop_teaming_provider_to_provider.append(filtered_chunk)


hop_teaming_provider_to_provider.to_csv('../data/hop_teaming_provider_to_provider.csv', index = False)

In [ ]:
cms_cols = ['NPI',
            'pri_spec',
            'sec_spec_all',
            'st',
            'org_nm',
            'hosp_afl_lbn_1',
            'hosp_afl_lbn_2',
            'hosp_afl_lbn_3',
            'hosp_afl_lbn_4',
            'hosp_afl_lbn_5']


cms = pd.read_csv('../data/National_Downloadable_File_2018_12.csv',
                  usecols = cms_cols, 
                  dtype = str, 
                  skipinitialspace = True,
                  chunksize = 100000,
                  low_memory = False)


cms_provider_data = pd.DataFrame()

for chunk in tqdm(cms):
    states = ['AL', 'AR', 'GA', 'KY', 'MO', 'MS', 'NC', 'TN', 'VA']
    filtered_chunk = chunk[cms_cols].query('st == @states').drop(columns = 'st')
    cms_provider_data = cms_provider_data.append(filtered_chunk)


cms_provider_data.columns = ['npi', 
                             'cms_pri_spec',
                             'cms_sec_spec_all',
                             'cms_org_nm',
                             'cms_hosp_afl_1', 
                             'cms_hosp_afl_2', 
                             'cms_hosp_afl_3',
                             'cms_hosp_afl_4', 
                             'cms_hosp_afl_5']


cms_provider_data = cms_provider_data.groupby('npi', as_index = False) \
    .agg({'cms_pri_spec': lambda row: '; '.join(set(row.dropna())),
          'cms_sec_spec_all': lambda row: '; '.join(set(row.dropna())),
          'cms_org_nm': lambda row: '; '.join(set(row.dropna())),
          'cms_hosp_afl_1': lambda row: '; '.join(set(row.dropna())),
          'cms_hosp_afl_2': lambda row: '; '.join(set(row.dropna())),
          'cms_hosp_afl_3': lambda row: '; '.join(set(row.dropna())),
          'cms_hosp_afl_4': lambda row: '; '.join(set(row.dropna())),
          'cms_hosp_afl_5': lambda row: '; '.join(set(row.dropna()))})


cms_provider_data.to_csv('../data/cms_provider_data.csv', index = False)

In [ ]:
hop_teaming_providers = regional_npis[regional_npis['npi'] \
    .isin(hop_teaming_provider_to_provider['from_npi'] \
    .append(hop_teaming_provider_to_provider['to_npi']) \
    .reset_index(drop = True))] \
    .drop(columns = ['organization_name', 'organization_name_other']) \
    .merge(cms_provider_data, how = 'left', on = 'npi')

hop_teaming_providers.to_csv('../data/hop_teaming_providers.csv', index = False)